# Practice Project Using ETL

Task: Extract data GDP of countries and transform GDP from million USD to billion USD, then push them to SQL database

## Importing libraries

In [21]:
#!python -m pip install -r requirement.txt

In [22]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import sqlite3

In [23]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'
table_attribs = ["CountryName", "GDP_USD_millions"]
csv_path = './world_gdp.csv'
table_name = 'Countries_by_GDP'
db_name = "World_Economies.db"

## Extracting Data

In [24]:
def extract(url, table_attribs):
    page = requests.get(url).text
    data = BeautifulSoup(page, 'html.parser')
    tables = data.find_all('tbody')
    rows = tables[2].find_all('tr') # index 2 of tables is GDP table
    df = pd.DataFrame(columns=table_attribs)
    for row in rows:
        cols = row.find_all('td')
        if len(cols) != 0:
            if cols[0].find('a') is not None and "—" not in cols[1] and "—" not in cols[2]:
                # print(cols[0].text, cols[1].text, sep = "\t")
                data_dict = {
                    "CountryName": cols[0].text.strip(),
                    "GDP_USD_millions": cols[1].text.strip()
                }
                per_frame = pd.DataFrame(data_dict, index=[0])
                df = pd.concat([df, per_frame], ignore_index=True)
    return df

df_gdp = extract(url, table_attribs)

In [25]:
df_gdp.head()

,CountryName,GDP_USD_millions
0,United States,"28,781,083"
1,China,"18,532,633"
2,Germany,"4,591,100"
3,Japan,"4,110,452"
4,India,"3,937,011"


## Transform Data

Convert Millions USD to Billions USD

In [26]:
def transform(df):
    GDP_columns = df["GDP_USD_millions"].to_list()
    GDP_columns = [float("".join(x.split(","))) for x in GDP_columns]
    GDP_columns = [np.round(x/1000, 2) for x in GDP_columns]
    df["GDP_USD_billions"] = GDP_columns
    df.drop(["GDP_USD_millions"], axis = 1, inplace=True)
    return df

df_gdp = transform(df_gdp)
df_gdp.head(5)

,CountryName,GDP_USD_billions
0,United States,28781.08
1,China,18532.63
2,Germany,4591.10
3,Japan,4110.45
4,India,3937.01


## Loading

In [27]:
def load_to_csv(df, csv_path):
    df.to_csv(csv_path)

def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

## Logging

In [28]:
def log_progress(message):
    time_stamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(time_stamp_format)
    with open("./etl_project_log.txt", "a") as f:
        f.write(timestamp + " : " + message + "\n")

## Function Calls

In [29]:
log_progress('Preliminaries complete. Initiating ETL process.')
df = extract(url, table_attribs)

log_progress('Data extraction complete. Initiating Transformation process.')
df = transform(df)

log_progress('Data extraction complete. Initiating loading process.')
load_to_csv(df, csv_path)

sql_connection = sqlite3.connect(db_name)
log_progress('SQL connection initiated.')

load_to_db(df, sql_connection, table_name)
log_progress('Data loaded to Database as table. Running a query')

query_statement = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
run_query(query_statement=query_statement, sql_connection=sql_connection)

log_progress('Process Complete.')

sql_connection.close()


SELECT * from Countries_by_GDP WHERE GDP_USD_billions >= 100
      CountryName  GDP_USD_billions
0   United States          28781.08
1           China          18532.63
2         Germany           4591.10
3           Japan           4110.45
4           India           3937.01
..            ...               ...
65      Guatemala            110.04
66           Oman            108.93
67       Bulgaria            107.93
68          Kenya            104.00
69      Venezuela            102.33

[70 rows x 2 columns]
